## 2021: Week 9 - Working with Strings

We have a guest contributor this week! Owen has just finished up his training at the Data School and is a regular Preppin' Data participator. So here's his challenge:

This challenge will be useful for anyone trying to improve their knowledge of Tableau Prep functions, and their data parsing in general. There is an opportunity to use ReGex in this challenge, but there is a longer workaround with other steps available. There will also be the chance to use LODs in this workflow.

We have been given a set of messy strings, which contain useful information that we need to connect to other datasets to eventually find out how much revenue we have generated by selling different products. This string provides us with information such as the quantity of items sold, the product ID code, the phone number of the buyer, and the area code which will let us find out where they are purchasing from. There will also be some small calculations needed to join certain datasets together!

### Input
- Customer Information
![img](https://1.bp.blogspot.com/-SMTJ0GxSp1U/YCKpJOLwQwI/AAAAAAAAAuQ/E3YyMQUTQZ4q_4-1o720lJYzdaqSml0CQCLcBGAsYHQ/w400-h205/2021W09.1.png)

- Area Code Lookup
![img2](https://1.bp.blogspot.com/-iZgM1VfejPg/YCKpJN_JseI/AAAAAAAAAuU/9CXJTVZ1C7YxnLGr33doeab33bVpxM8IgCLcBGAsYHQ/w400-h356/2021W09.2.png)

- Product Lookup
![img3](https://1.bp.blogspot.com/-Vyn5REjLtjE/YCKpJf1iCbI/AAAAAAAAAuY/UR7V8B_Mz54thsX5yfd08fBqI9rPdWbxQCLcBGAsYHQ/s0/2021W09.3.png)

### Requirement
- Input the Customer Information file, split the values and reshape the data so there is a separate ID on each row.
- Each ID field contains the following information we need to extract:
    - The first 6 digits present in each ID is the customers phone number
    - The first 2 digits after the ‘,’ is the last 2 digits of the area code 
    - The letter following this is the first letter of the name of the area that they are calling from
    - The digits after this letter resemble the quantity of products ordered
    - The letters after the ‘-‘ are the product ID codes 
- Rename these fields appropriately, and remove any unwanted columns – leaving only these 5 columns in the workflow. 
- Input the Area Code Lookup Table – find a way to join it to the Customer information file 
- We don’t actually sell products in Clevedon, Fakenham, or Stornoway. Exclude these from our dataset
- In some cases, the ID field does not provide accurate enough conditions to know where the customer is from. Exclude any phone numbers where the join has produced duplicated records.
- Remove any unwanted fields created from the join. 
- Join this dataset to our product lookup table. 
- For each area, and product, find the total sales values, rounded to zero decimal places 
- Rank how well each product sold in each area.
- For each area, work out the percent of total that each different product contributes to the overall revenue of that Area, rounded to 2 decimal places. 
- Output the data

### Output
5 fields
- Rank
- Area
- Product Name
- Revenue
- % of Total – Product

144 rows (145 including headers)

![img](https://1.bp.blogspot.com/-dzh6MW8N5SE/YD_NaLgKbPI/AAAAAAAAAw8/ArUJI4cdV4Ia-dkjhDi5wRUBcFEOYFcSgCLcBGAsYHQ/s320/image.png)

In [399]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

### Input the Customer Information file, split the values and reshape the data  so there is a separate ID on each row

In [400]:
customer = pd.read_excel("./data/Customer Information.xlsx")
customer.shape

(200, 1)

In [401]:
customer

,IDs
0,"Ju856452,13S24-SPL wd234175,29M77-SPL KZ621372..."
1,"jM391563,00C69-SPL Uc296328,17S73-SBP EL580409..."
2,"rV469041,02L68-HS Rn519453,20L22-SPL pd615208,..."
3,"GQ505960,03W64-SBP JS186662,22M1-SBP Id680462,..."
4,"bf677129,05D99-SBP MA755072,24A76-SBP Zf805822..."
...,...
195,"nB634036,03W39-SBP Hg244254,22M64-SPL LD463660..."
196,"Ya795584,05D71-SPL TE924789,24A9-SBP rf761703,..."
197,"oi598659,06H99-HS zs924627,25D2-SPL XL027397,3..."
198,"JI221536,07M95-SBP hz193964,26S45-SBP gq663772..."


In [402]:
customer["info"] = customer["IDs"].map(lambda x: x.split(" "))

In [403]:
customer_info = customer["info"].apply(pd.Series).stack().reset_index(drop=True)
customer_info = pd.DataFrame(customer_info, columns=["IDs"])
customer_info.shape

(999, 1)

In [404]:
customer_info.head()

,IDs
0,"Ju856452,13S24-SPL"
1,"wd234175,29M77-SPL"
2,"KZ621372,42K26-SBP"
3,"AY559207,53K50-HS"
4,"jM391563,00C69-SPL"
...,...
994,"Tf723726,09H15-SBP"
995,"et098928,28F45-SPL"
996,"Nr833098,41A65-HS"
997,"Md961094,52A25-SBP"


### Each ID field contains the following information we need to extract

In [405]:
customer_info["phone_number"] = customer_info["IDs"].map(lambda x: x.split(" ")[0].split(",")[0][-6:])
customer_info["area_code"] = customer_info["IDs"].map(lambda x: x.split(" ")[0].split(",")[1][:2])
customer_info["area_first_letter"] = customer_info["IDs"].map(lambda x: x.split(" ")[0].split(",")[1][2])
customer_info["order_quantity"] = customer_info["IDs"].map(lambda x: x.split(" ")[0].split(",")[1].split("-")[0][2:][1:])
customer_info["product_id"] = customer_info["IDs"].map(lambda x: x.split(" ")[0].split(",")[1].split("-")[-1])
customer_info.head()

,IDs,phone_number,area_code,area_first_letter,order_quantity,product_id
0,"Ju856452,13S24-SPL",856452,13,S,24,SPL
1,"wd234175,29M77-SPL",234175,29,M,77,SPL
2,"KZ621372,42K26-SBP",621372,42,K,26,SBP
3,"AY559207,53K50-HS",559207,53,K,50,HS
4,"jM391563,00C69-SPL",391563,00,C,69,SPL


### Rename these fields appropriately, and remove any unwanted columns

In [407]:
customer_info = customer_info.drop(["IDs"], axis=1)
customer_info.head()

,phone_number,area_code,area_first_letter,order_quantity,product_id
0,856452,13,S,24,SPL
1,234175,29,M,77,SPL
2,621372,42,K,26,SBP
3,559207,53,K,50,HS
4,391563,00,C,69,SPL


### Input the Area Code Lookup Table – find a way to join it to the Customer information

In [408]:
area_code_lookup = pd.read_excel("./data/Area Code Lookup.xlsx", dtype="object")
area_code_lookup.info()

,Code,Area
0,0114,Sheffield
1,0115,Nottingham
2,0116,Leicester
3,0117,Bristol
4,0118,Reading
...,...,...
748,02894,Antrim
749,02895,Belfast
750,02896,Belfast
751,02897,Saintfield


In [409]:
area_code = area_code_lookup["Code"].map(lambda x: x[-2:])
first_letter = area_code_lookup["Area"].map(lambda x: x[0])
area_code = area_code + first_letter
area_code_lookup["area_code"] = area_code
area_code_lookup.head()

,Code,Area,area_code
0,0114,Sheffield,14S
1,0115,Nottingham,15N
2,0116,Leicester,16L
3,0117,Bristol,17B
4,0118,Reading,18R
...,...,...,...
748,02894,Antrim,94A
749,02895,Belfast,95B
750,02896,Belfast,96B
751,02897,Saintfield,97S


In [411]:
customer_info["code_area"] = customer_info["area_code"] + customer_info["area_first_letter"]
customer_info.shape

,phone_number,area_code,area_first_letter,order_quantity,product_id,code_area
0,856452,13,S,24,SPL,13S
1,234175,29,M,77,SPL,29M
2,621372,42,K,26,SBP,42K
3,559207,53,K,50,HS,53K
4,391563,00,C,69,SPL,00C
...,...,...,...,...,...,...
994,723726,09,H,15,SBP,09H
995,098928,28,F,45,SPL,28F
996,833098,41,A,65,HS,41A
997,961094,52,A,25,SBP,52A


In [412]:
customer_info = (customer_info.merge(area_code_lookup, how="left", left_on="code_area", right_on="area_code")
                              .drop(["area_code_y"], axis=1)
                              .rename(columns={"area_code_x": "last_digit_area"}))
customer_info.shape

,phone_number,last_digit_area,area_first_letter,order_quantity,product_id,code_area,Code,Area
0,856452,13,S,24,SPL,13S,018513,Stornoway
1,234175,29,M,77,SPL,29M,01629,Matlock
2,621372,42,K,26,SBP,42K,01542,Keith
3,621372,42,K,26,SBP,42K,02842,Kircubbin
4,559207,53,K,50,HS,53K,01553,Kings Lynn
...,...,...,...,...,...,...,...,...
1241,098928,28,F,45,SPL,28F,01328,Fakenham
1242,833098,41,A,65,HS,41A,01241,Arbroath
1243,961094,52,A,25,SBP,52A,01252,Aldershot
1244,961094,52,A,25,SBP,52A,019752,Alford (Aberdeen)


### We don’t actually sell products in Clevedon, Fakenham, or Stornoway. Exclude these from our dataset

In [413]:
index_to_del = customer_info[customer_info["Area"].isin(["Clevedon", "Fakenham", "Stornoway"])].index
print(len(index_to_del))

customer_info = customer_info.drop(index_to_del, axis=0)
customer_info

63


,phone_number,last_digit_area,area_first_letter,order_quantity,product_id,code_area,Code,Area
1,234175,29,M,77,SPL,29M,01629,Matlock
2,621372,42,K,26,SBP,42K,01542,Keith
3,621372,42,K,26,SBP,42K,02842,Kircubbin
4,559207,53,K,50,HS,53K,01553,Kings Lynn
5,391563,00,C,69,SPL,00C,01200,Clitheroe
...,...,...,...,...,...,...,...,...
1240,723726,09,H,15,SBP,09H,01409,Holsworthy
1242,833098,41,A,65,HS,41A,01241,Arbroath
1243,961094,52,A,25,SBP,52A,01252,Aldershot
1244,961094,52,A,25,SBP,52A,019752,Alford (Aberdeen)


### In some cases, the ID field does not provide accurate enough conditions to know where the customer is from. Exclude any phone numbers where the join has produced duplicated records

In [414]:
customer_info = customer_info.drop_duplicates(subset=["phone_number"], keep=False)
customer_info.shape

,phone_number,last_digit_area,area_first_letter,order_quantity,product_id,code_area,Code,Area
1,234175,29,M,77,SPL,29M,01629,Matlock
4,559207,53,K,50,HS,53K,01553,Kings Lynn
8,296328,17,S,73,SBP,17S,01917,Sunderland
9,580409,31,E,18,SBP,31E,0131,Edinburgh
12,961467,54,U,1,SPL,54U,01854,Ullapool
...,...,...,...,...,...,...,...,...
1238,752977,50,C,90,HS,50C,01650,Cemmaes Road
1239,414564,63,A,65,HS,63A,01863,Ardgay
1240,723726,09,H,15,SBP,09H,01409,Holsworthy
1242,833098,41,A,65,HS,41A,01241,Arbroath


### Remove any unwanted fields created from the join. 

In [415]:
customer_info = customer_info.drop(["last_digit_area", "area_first_letter", "code_area"], axis=1)
customer_info

,phone_number,order_quantity,product_id,Code,Area
1,234175,77,SPL,01629,Matlock
4,559207,50,HS,01553,Kings Lynn
8,296328,73,SBP,01917,Sunderland
9,580409,18,SBP,0131,Edinburgh
12,961467,1,SPL,01854,Ullapool
...,...,...,...,...,...
1238,752977,90,HS,01650,Cemmaes Road
1239,414564,65,HS,01863,Ardgay
1240,723726,15,SBP,01409,Holsworthy
1242,833098,65,HS,01241,Arbroath


### Join this dataset to our product lookup table.

In [416]:
product_lookup = pd.read_excel("./data/Product Lookup.xlsx")
product_lookup["Price"] = product_lookup["Price"].map(lambda x: x[1:])
product_lookup["Price"] = product_lookup["Price"].astype(float)

customer_info = customer_info.merge(product_lookup, how="left", 
                                    left_on="product_id", right_on="Product ID")
customer_info = customer_info.drop(["Product ID"], axis=1)
customer_info.shape

,phone_number,order_quantity,product_id,Code,Area,Product Name,Price
0,234175,77,SPL,01629,Matlock,"Soap, Liquid",6.50
1,559207,50,HS,01553,Kings Lynn,Hand Sanitiser,2.29
2,296328,73,SBP,01917,Sunderland,"Soap, Bar",4.55
3,580409,18,SBP,0131,Edinburgh,"Soap, Bar",4.55
4,961467,1,SPL,01854,Ullapool,"Soap, Liquid",6.50
...,...,...,...,...,...,...,...
746,752977,90,HS,01650,Cemmaes Road,Hand Sanitiser,2.29
747,414564,65,HS,01863,Ardgay,Hand Sanitiser,2.29
748,723726,15,SBP,01409,Holsworthy,"Soap, Bar",4.55
749,833098,65,HS,01241,Arbroath,Hand Sanitiser,2.29


### For each area, and product, find the total sales values, rounded to zero decimal places

In [417]:
customer_info["order_quantity"] = customer_info["order_quantity"].astype(int)
customer_info["Total_Sales"] = customer_info["Price"] * customer_info["order_quantity"]
customer_info

,phone_number,order_quantity,product_id,Code,Area,Product Name,Price,Total_Sales
0,234175,77,SPL,01629,Matlock,"Soap, Liquid",6.50,500.50
1,559207,50,HS,01553,Kings Lynn,Hand Sanitiser,2.29,114.50
2,296328,73,SBP,01917,Sunderland,"Soap, Bar",4.55,332.15
3,580409,18,SBP,0131,Edinburgh,"Soap, Bar",4.55,81.90
4,961467,1,SPL,01854,Ullapool,"Soap, Liquid",6.50,6.50
...,...,...,...,...,...,...,...,...
746,752977,90,HS,01650,Cemmaes Road,Hand Sanitiser,2.29,206.10
747,414564,65,HS,01863,Ardgay,Hand Sanitiser,2.29,148.85
748,723726,15,SBP,01409,Holsworthy,"Soap, Bar",4.55,68.25
749,833098,65,HS,01241,Arbroath,Hand Sanitiser,2.29,148.85


In [418]:
grouped_total_sales = customer_info.groupby(["Area", "Product Name"])["Total_Sales"].sum().round(0).reset_index()
grouped_total_sales

,Area,Product Name,Total_Sales
0,Aberdeen,Hand Sanitiser,353.0
1,Aberdeen,"Soap, Bar",2507.0
2,Aberdeen,"Soap, Liquid",793.0
3,Arbroath,Hand Sanitiser,847.0
4,Arbroath,"Soap, Bar",687.0
...,...,...,...
139,Worthing,"Soap, Bar",2020.0
140,Worthing,"Soap, Liquid",1131.0
141,Wrexham,Hand Sanitiser,545.0
142,Wrexham,"Soap, Bar",2129.0


### Rank how well each product sold in each area

In [419]:
grouped_total_sales["Rank"] = grouped_total_sales.groupby(["Area"])["Total_Sales"].rank(ascending=False)
grouped_total_sales["Rank"] = grouped_total_sales["Rank"].astype(int)
grouped_total_sales

,Area,Product Name,Total_Sales,Rank
0,Aberdeen,Hand Sanitiser,353.0,3
1,Aberdeen,"Soap, Bar",2507.0,1
2,Aberdeen,"Soap, Liquid",793.0,2
3,Arbroath,Hand Sanitiser,847.0,2
4,Arbroath,"Soap, Bar",687.0,3
...,...,...,...,...
139,Worthing,"Soap, Bar",2020.0,1
140,Worthing,"Soap, Liquid",1131.0,2
141,Wrexham,Hand Sanitiser,545.0,3
142,Wrexham,"Soap, Bar",2129.0,1


### For each area, work out the percent of total that each different product contributes to the overall revenue of that Area, rounded to 2 decimal places

In [420]:
# Calculation for Total_Sales per Area
grouped = customer_info.groupby(["Area", "Product Name"])["Total_Sales"].sum().reset_index()
area_total = grouped.groupby(["Area"])["Total_Sales"].sum().reset_index()
area_total.columns = ["Area", "Area_Total"]

In [421]:
# Merging the Area_Total Values to grouped dataframe
grouped_total_sales = grouped_total_sales.merge(area_total, how="left", on="Area")
grouped_total_sales

,Area,Product Name,Total_Sales,Rank,Area_Total
0,Aberdeen,Hand Sanitiser,353.0,3,3652.71
1,Aberdeen,"Soap, Bar",2507.0,1,3652.71
2,Aberdeen,"Soap, Liquid",793.0,2,3652.71
3,Arbroath,Hand Sanitiser,847.0,2,3958.85
4,Arbroath,"Soap, Bar",687.0,3,3958.85
...,...,...,...,...,...
139,Worthing,"Soap, Bar",2020.0,1,3485.54
140,Worthing,"Soap, Liquid",1131.0,2,3485.54
141,Wrexham,Hand Sanitiser,545.0,3,3337.42
142,Wrexham,"Soap, Bar",2129.0,1,3337.42


In [422]:
grouped_total_sales["% of Total - Product"] = grouped_total_sales["Total_Sales"] / grouped_total_sales["Area_Total"] * 100
grouped_total_sales["% of Total - Product"] = grouped_total_sales["% of Total - Product"].round(2)
grouped_total_sales["Total_Sales"] = grouped_total_sales["Total_Sales"].astype(int)

In [423]:
grouped_total_sales = grouped_total_sales.rename(columns={"Total_Sales" : "Revenue"})
grouped_total_sales = grouped_total_sales.drop(["Area_Total"], axis=1)
grouped_total_sales = grouped_total_sales.loc[:, ["Rank", "Area", "Product Name", "Revenue", "% of Total - Product"]]
grouped_total_sales.head()

,Rank,Area,Product Name,Revenue,% of Total - Product
0,3,Aberdeen,Hand Sanitiser,353,9.66
1,1,Aberdeen,"Soap, Bar",2507,68.63
2,2,Aberdeen,"Soap, Liquid",793,21.71
3,2,Arbroath,Hand Sanitiser,847,21.40
4,3,Arbroath,"Soap, Bar",687,17.35


### Output the data

In [424]:
grouped_total_sales.to_csv("./output/Week9_output.csv")